In [1]:
import numpy as np
import pandas as pd

In [2]:
# reading in external data
# puma_acs_data = pd.read_csv("data/ACS_2017.csv")
# puma_lodes_data = pd.read_csv("data/wac_puma_2017.csv")

puma_acs_data = pd.read_csv("data/ACS_2018.csv")
puma_lodes_data = pd.read_csv("data/wac_puma.csv")

puma_acs_data["PUMA"] = puma_acs_data["PUMA"].astype(str).str.zfill(7)
puma_acs_data = puma_acs_data.set_index("PUMA")

puma_lodes_data["puma"] = puma_lodes_data["puma"].astype(str).str.zfill(7)
puma_lodes_data = puma_lodes_data.set_index("puma")

In [3]:
# # reading in the individual PUMS
# df_a = pd.read_csv("data/psam_pusa_2017.csv")
# df_b = pd.read_csv("data/psam_pusb_2017.csv")

In [4]:
# reading in the individual PUMS
df_a = pd.read_csv("data/psam_pusa.csv")
df_b = pd.read_csv("data/psam_pusb.csv")

In [5]:
# combinining the two parts of the indivdiual PUMS
df_p = pd.concat([df_a, df_b])

In [6]:
# # reading in the household PUMS for referencing purposes
# dfh_a = pd.read_csv("data/psam_husa_2017.csv")
# dfh_b = pd.read_csv("data/psam_husb_2017.csv")

In [7]:
# reading in the household PUMS for referencing purposes
dfh_a = pd.read_csv("data/psam_husa.csv")
dfh_b = pd.read_csv("data/psam_husb.csv")

In [8]:
# combining the two parts of the household PUMS
df_h = pd.concat([dfh_a, dfh_b])

In [9]:
# cleaning missing identifier data
df_p["MIGPUMA"] = df_p["MIGPUMA"].fillna(0)
df_p["MIGSP"] = df_p["MIGSP"].fillna(0)

In [10]:
# filtering out the PUMS to people 18+ who moved from places in the contiguous united states to other places in the contiguous united states
print(df_p.shape)
df_subset = df_p[df_p["AGEP"].ravel() >= 18]
print(df_subset.shape)
# 2 and 15 correspond to Alaska and Hawaii
df_subset = df_subset[(df_subset["MIGSP"].ravel() <= 56) & (df_subset["MIGSP"].ravel() != 2) & (df_subset["MIGSP"].ravel() != 15)]
print(df_subset.shape)
df_subset = df_subset[(df_subset["ST"].ravel() != 2) & (df_subset["ST"].ravel() != 15)]
print(df_subset.shape)

(3214539, 286)
(2562591, 286)
(2545881, 286)
(2530899, 286)


In [11]:
# converting MIGPUMA and PUMA values to unique values
df_subset["MIGPUMA_st"] = df_subset["MIGSP"].astype(int).astype(str).str.zfill(2) + df_subset["MIGPUMA"].astype(int).astype(str).str.zfill(5)
df_subset["PUMA_st"] = df_subset["ST"].astype(str).str.zfill(2) + df_subset["PUMA"].astype(str).str.zfill(5)

In [12]:
df_subset["MIGPUMA_st"].value_counts()

0000000    2202915
0603700       8978
1703400       4911
2500390       4785
0400100       4617
            ...   
2201600         67
2101000         66
4806900         64
1702601         62
5401300         60
Name: MIGPUMA_st, Length: 976, dtype: int64

In [13]:
df_subset["PUMA_st"].value_counts()

0102500    3609
5310200    3393
5500100    3256
5500700    3183
1200500    2778
           ... 
2701403     430
4804633     429
4203207     421
5541001     417
2701402     412
Name: PUMA_st, Length: 2336, dtype: int64

In [14]:
# skipping sampling
df = df_subset

In [15]:
# including some household data into the individual data
child = df_h[["NP", "TYPE", "TEN", "VALP", "VEH", "FES", "FINCP", "FPARC", "GRNTP", "GRPIP", "HHT", "HINCP", "OCPIP", "PARTNER", "R18", "SMOCP", "WIF", "HUPAOC", "SERIALNO"]].groupby("SERIALNO").first()
df = pd.merge(df, child, left_on="SERIALNO", right_index=True, how="left")
df["child"] = np.where(df["HUPAOC"] == 4, 0, 1)
df["child"]

0          1
1          1
2          1
3          1
4          1
          ..
1566019    0
1566020    0
1566024    0
1566025    0
1566026    0
Name: child, Length: 2530899, dtype: int32

In [16]:
df["child"].value_counts()

0    1755972
1     774927
Name: child, dtype: int64

In [17]:
df["FER_CL"] = df["FER"].fillna(0)
df["FER_CL"] = np.where(df["FER_CL"] == 2, 0, df["FER_CL"])
df["FER_CL"].value_counts()

0.0    2496139
1.0      34760
Name: FER_CL, dtype: int64

In [18]:
rec_child = df.groupby("SERIALNO")["FER_CL"].max()

In [19]:
df["REC_CHILD"] = rec_child.loc[df["SERIALNO"]].values 

In [20]:
# reading in MIGPUMA-PUMA distance data
distances = pd.read_csv("data/puma_distance_matrix.csv").set_index("Unnamed: 0")
distances.index = distances.index.astype(str).str.zfill(7)
distances.columns = distances.columns.map(str).str.zfill(7)
distances

,0600105,0600102,0608502,0600108,0600107,0600101,0600110,0608504,0600109,0600106,...,2602200,2600900,4702402,4702300,4702401,4700500,4702501,5310400,5310300,5310100
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0600100,0,0,39057,0,0,0,0,32785,0,0,...,3659560,3658028,3704534,3707792,3697348,3709156,3675491,1510819,1366000,1521187
0600700,267650,254154,308889,293346,284521,247767,274332,302616,290360,273847,...,3548479,3546947,3757628,3748449,3750726,3741417,3716148,1293877,1149057,1304245
0601100,342063,328567,383302,367759,358934,322180,361853,377029,364773,348259,...,3646878,3645346,3856026,3846847,3849125,3839816,3814546,1246358,1101538,1256725
0601300,54185,44346,90212,80011,67520,45446,68763,83940,71683,55170,...,3635231,3633699,3743085,3746342,3735899,3747707,3714042,1486490,1341670,1496858
0601500,487278,473782,528517,512974,504149,467395,507068,522244,509987,493474,...,3594176,3592644,3803325,3794145,3796423,3787114,3761844,1149082,1004262,1159450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304007,3996091,4004766,3970737,3980928,3987472,4012807,3958553,3961958,3991797,3985390,...,1210028,1269788,373179,429406,391176,457262,418124,4143960,4251644,4527073
5151000,4385759,4394434,4360405,4370596,4377141,4402476,4348221,4351626,4381466,4375058,...,917732,1041456,718843,663499,695569,705881,692139,4231226,4338910,4661344
5151001,4543694,4530198,4542272,4569390,4560565,4523811,4547511,4533493,4566404,4549890,...,1050887,1169711,900710,845366,877436,887748,874006,4350356,4458040,4780474


In [21]:
# adding stay field
df["STAY"] = np.where(df["MIGPUMA_st"] == "0000000", 1, 0)

In [22]:
df["STAY"].value_counts()

1    2202915
0     327984
Name: STAY, dtype: int64

In [23]:
# naics_key = pd.read_csv("data/puma_naics_2017.txt", sep=" ", header=None).set_index(0)
naics_key = pd.read_csv("data/puma_naics.txt", sep=" ", header=None).set_index(0)
naics_key = naics_key.T.to_dict("list")
naics_key = {key:value[0] for key, value in zip(naics_key.keys(), naics_key.values())}
naics_key

{'111': '.AGR-Crop',
 '112': '.AGR-Animal',
 '1133': '.AGR-Logging',
 '113M': '.AGR-Forestry',
 '114': '.AGR-Fishing,',
 '115': '.AGR-Support',
 '211': '.EXT-Oil',
 '2121': '.EXT-Coal',
 '2122': '.EXT-Metal',
 '2123': '.EXT-Nonmetallic',
 '213': '.EXT-Support',
 '2211P': '.UTL-Electric',
 '2212P': '.UTL-Natural',
 '22132': '.UTL-Sewage',
 '2213M': '.UTL-Water,',
 '221MP': '.UTL-Electric',
 '22S': '.UTL-Not',
 '23': '.CON-Construction',
 '3113': '.MFG-Sugar',
 '3114': '.MFG-Fruit',
 '3115': '.MFG-Dairy',
 '3116': '.MFG-Animal',
 '311811': '.MFG-Retail',
 '3118Z': '.MFG-Bakeries',
 '311M1': '.MFG-Animal',
 '311M2': '.MFG-Seafood',
 '311S': '.MFG-Not',
 '3121': '.MFG-Beverage',
 '3122': '.MFG-Tobacco',
 '3131': '.MFG-Fiber,',
 '3132Z': '.MFG-Fabric',
 '3133': '.MFG-Textile',
 '31411': '.MFG-Carpet',
 '314Z': '.MFG-Textile',
 '315M': '.MFG-Cut',
 '3162': '.MFG-Footwear',
 '316M': '.MFG-Leather',
 '31M': '.MFG-Knitting',
 '3211': '.MFG-Sawmills',
 '3212': '.MFG-Veneer,',
 '32199M': '.MFG-Pr

In [24]:
df["NAICS"] = df["NAICSP"].map(naics_key)
df["NAICS"] = df["NAICS"].str.strip(".,").str[0:3]

In [25]:
df["NAICS"] = df["NAICS"].replace("Une", np.NaN).replace("MIL", np.NaN)

In [26]:
df["NAICS"].value_counts()

MED    247609
RET    200347
EDU    186754
MFG    186553
PRF    135138
ADM    130881
FOD    122090
CON    116691
SRV     91802
TRN     81404
FIN     80926
WHL     44625
ENT     43916
INF     35638
REL     35118
PUB     34602
AGR     29865
UTL     15897
EXT      9428
MNG      2905
Name: NAICS, dtype: int64

In [27]:
df[(df["NAICS"].isna()) & ~(df["NAICSP"].isna())][["NAICS", "NAICSP"]]

,NAICS,NAICSP
14,NaN,928110P1
64,NaN,928110P1
95,NaN,928110P1
344,NaN,928110P1
397,NaN,999920
...,...,...
1565292,NaN,999920
1565444,NaN,928110P2
1565615,NaN,999920
1565889,NaN,928110P2


In [28]:
df["NAICSP"].value_counts()

6111     119254
23       116691
722Z     100863
622M      81517
611M1     55666
          ...  
22S         210
31M         207
3133        199
3131        159
33MS        150
Name: NAICSP, Length: 270, dtype: int64

In [29]:
# renaming to make chosen destination/origin distinction clear
df = df.rename(columns={"PUMA_st": "CHOSEN", "MIGPUMA_st": "ORIGIN"})

In [30]:
# reading in PUMA-MIGPUMA equivalency data
puma_migpuma = pd.read_excel("data/puma_migpuma.xlsx", dtype="str")
puma_migpuma["PUMA"] = puma_migpuma["State"] + puma_migpuma["PUMA"]
puma_migpuma["MIGPUMA"] = puma_migpuma["State"] + puma_migpuma["MIGPUMA"]
# some MIGPUMA fields have footnotes
puma_migpuma["MIGPUMA"] = puma_migpuma["MIGPUMA"].str.strip("*")
puma_migpuma = puma_migpuma.set_index("PUMA")
puma_migpuma.head()

,State,MIGPUMA
PUMA,,
0100100,01,0100190
0100200,01,0100290
0100301,01,0100290
0100302,01,0100290
0100400,01,0100400


In [31]:
df["ORIGIN"] = np.where(df["ORIGIN"] == "0000000", puma_migpuma.loc[df["CHOSEN"], "MIGPUMA"], df["ORIGIN"])

In [32]:
import random
random.seed(3)
random.choice(list(distances.columns))

'0800802'

In [33]:
# creating some new fields
puma_acs_data["UNEMP"] = puma_acs_data["Population 16 Years and Over in Labor Force Civilian Unemployed"] / puma_acs_data["Population 16 Years and Over in Labor Force Civilian"]
puma_acs_data["COLLEGE"] = puma_acs_data["Population 3 Years and Over Enrolled in School Private School College"] + puma_acs_data["Population 3 Years and Over Enrolled in School Public School College"]
puma_acs_data["VACANCY_PCT"] = puma_acs_data["Housing Units Vacant"] / puma_acs_data["Housing Units"]

In [34]:
# removing duplicate indices/adding normal index
df = df.reset_index().drop(["index"], axis=1)

In [35]:
def isNaN(num):
    return num != num

In [36]:
cbsas = pd.read_csv("data/puma_density.csv")
cbsas["GEOID"] = cbsas["GEOID"].astype(str).str.zfill(7)
cbsas = cbsas.set_index("GEOID")
cbsas

,Unnamed: 0,GISMATCH,GISJOIN,STATEFIP,State,PUMA,Name,center,type,CBSA_name
GEOID,,,,,,,,,,
0600105,0,600105,G06000105,6,California,105,"Alameda County (West)--San Leandro, Alameda & ...",POINT (-2258808.793332302 339396.4722123478),T34,"San Francisco-Oakland-Hayward, CA Metro Area"
0600102,1,600102,G06000102,6,California,102,Alameda County (Northwest)--Oakland (Northwest...,POINT (-2262119.20387295 349529.9544024405),T34,"San Francisco-Oakland-Hayward, CA Metro Area"
0608502,2,608502,G06008502,6,California,8502,Santa Clara County (Northwest)--Sunnyvale & Sa...,POINT (-2251625.280558518 299700.2119385958),Metro,"San Jose-Sunnyvale-Santa Clara, CA Metro Area"
0600108,3,600108,G06000108,6,California,108,"Alameda County (Southwest)--Union City, Newark...",POINT (-2249381.571782368 314454.5304018197),T34,"San Francisco-Oakland-Hayward, CA Metro Area"
0600107,4,600107,G06000107,6,California,107,Alameda County (Central)--Hayward City PUMA,POINT (-2249209.156361292 325491.2460234718),T34,"San Francisco-Oakland-Hayward, CA Metro Area"
...,...,...,...,...,...,...,...,...,...,...
4700500,2373,4700500,G47000500,47,Tennessee,500,Sumner County--Hendersonville City PUMA,POINT (845555.1346444382 -73006.03095774697),Metro,"Nashville-Davidson--Murfreesboro--Franklin, TN..."
4702501,2374,4702501,G47002501,47,Tennessee,2501,Nashville-Davidson (East) PUMA,POINT (834713.9667902931 -111417.8817361052),Metro,"Nashville-Davidson--Murfreesboro--Franklin, TN..."
5310400,2375,5310400,G53010400,53,Washington,10400,"Stevens, Okanogan, Pend Oreille & Ferry Counti...",POINT (-1684664.90787742 1429380.26889712),NaN,NaN


In [37]:
cbsas["type"].value_counts()

T34      1129
Metro     881
Micro     184
Name: type, dtype: int64

In [38]:
cbsas["type_num"] = 3
cbsas["type_num"] = np.where(cbsas["type"] == "T34", 0, cbsas["type_num"])
cbsas["type_num"] = np.where(cbsas["type"] == "Metro", 1, cbsas["type_num"])
cbsas["type_num"] = np.where(cbsas["type"] == "Micro", 2, cbsas["type_num"])

In [39]:
cbsas["Name_num"] = pd.factorize(cbsas["CBSA_name"])[0]
cbsas["Name_num"]

GEOID
0600105      0
0600102      0
0608502      1
0600108      0
0600107      0
          ... 
4700500    283
4702501    283
5310400     -1
5310300    536
5310100    537
Name: Name_num, Length: 2378, dtype: int64

In [40]:
puma_migpuma = puma_migpuma.dropna()
puma_migpuma["State"] = puma_migpuma["State"].astype(int)

In [41]:
import time

In [42]:
def print_time():
    temp = time.time()
    print(temp)

In [43]:
df_ext = df

In [44]:
# # adding time and distance data for each one
# for migpuma in distances.index:
#     for puma in distances.index:
#         dist = distances.loc[migpuma, puma]
        
#     for i in range(1, 101):
#         key = "ALT" + str(i) + "_"
#         puma = df_ext.loc[migpuma, ]
#         dist = distance_matrix.loc[migpuma, puma]
#         time = time_matrix.loc[migpuma, puma]
#         df_ext.loc[df_subset["ORIGIN"] == migpuma, key + "TIME"] = time_matrix.loc[migpuma, puma]
#         df_subset.loc[df_subset["ORIGIN"] == migpuma, key + "DIST"] = distance_matrix.loc[migpuma, puma]
#         if dist == 0:
#             df_subset.loc[df_subset["ORIGIN"] == migpuma, key + "INTERNAL_MOVE"] = 1

In [45]:
# migpuma_acs_data = pd.read_csv("data/ACS_MIGPUMA_2017_new.csv").drop("Geo_FIPS", axis=1)
# migpuma_lodes_data = pd.read_csv("data/wac_migpuma_2017.csv")

# migpuma_acs_data["MIGPUMA"] = migpuma_acs_data["MIGPUMA"].astype(str).str.zfill(7)
# migpuma_acs_data = migpuma_acs_data.rename({"Total Population": "TOT_POP", "Population Density (Per Sq. Mile)": "DENS", "Total Population Under 18 Years": "POP_UND_18", "Total Population 18 to 34 Years": "POP_18_34", "Total Population 35 to 64 Years": "POP_35_64", "Total Population 65 and Over": "POP_OVER_65", "Median Age": "MED_AGE", "Population 25 Years and Over Less than High School": "EDU_NOHIGH", "Population 25 Years and Over Some College": "EDU_SOMECOL", "Population 25 Years and Over Bachelors Degree": "EDU_BACH", "Population 25 Years and Over Masters Degree": "EDU_MASTERS", "Population 25 Years and Over Professional School Degree": "EDU_PROF", "Population 25 Years and Over Doctorate Degree": "EDU_PHD", "Population 3 Years and Over Enrolled in School Public School College": "ENRL_PUB_COLL", "Population 3 Years and Over Enrolled in School Private School College": "ENRL_PRI_COLL", "Gini Index": "GINI", "SE_A14022_001": "AGG_INC", "Median Household Income (In 2017 Inflation Adjusted Dollars)": "HH_MED_INC", "Occupied Housing Units": "HU_OCC", "Occupied Housing Units Owner Occupied": "HU_OCC_OWN", "Occupied Housing Units Renter Occupied": "HU_OCC_RENT", "Housing Units": "HU_TOT", "Housing Units Vacant": "HU_VAC", "Median Value": "HH_MED_VAL", "Median Gross Rent": "HH_MED_RENT", "Median Gross Rent as a Percentage of Household  Income": "HH_MED_RENT_PCT_INC", "Median Selected Monthly Owner Costs as a  Percentage of Household Income": "HH_MED_OWN_PCT_INC", "Total Population Foreign Born": "FOREIGN_BORN", "Population 25 Years and Over High School Graduate (Includes Equivalency)": "EDU_HIGH", "Population 16 Years and Over in Labor Force in Armed Forces": "MIL_NUM"}, axis=1)

In [46]:
# reading in migpuma data and renaming them to more palatable keys
migpuma_acs_data = pd.read_csv("data/ACS_MIGPUMA_2018.csv").drop("Geo_FIPS", axis=1)
migpuma_lodes_data = pd.read_csv("data/wac_migpuma.csv")

migpuma_acs_data["MIGPUMA"] = migpuma_acs_data["MIGPUMA"].astype(str).str.zfill(7)
migpuma_acs_data = migpuma_acs_data.rename({"Total Population": "TOT_POP", "Population Density (Per Sq. Mile)": "DENS", "Total Population Under 18 Years": "POP_UND_18", "Total Population 18 to 34 Years": "POP_18_34", "Total Population 35 to 64 Years": "POP_35_64", "Total Population 65 and Over": "POP_OVER_65", "Median Age": "MED_AGE", "Population 25 Years and Over Less than High School": "EDU_NOHIGH", "Population 25 Years and Over Some College": "EDU_SOMECOL", "Population 25 Years and Over Bachelors Degree": "EDU_BACH", "Population 25 Years and Over Masters Degree": "EDU_MASTERS", "Population 25 Years and Over Professional School Degree": "EDU_PROF", "Population 25 Years and Over Doctorate Degree": "EDU_PHD", "Population 3 Years and Over Enrolled in School Public School College": "ENRL_PUB_COLL", "Population 3 Years and Over Enrolled in School Private School College": "ENRL_PRI_COLL", "Gini Index": "GINI", "SE_A14022_001": "AGG_INC", "Median Household Income (In 2019 Inflation Adjusted Dollars)": "HH_MED_INC", "Occupied Housing Units": "HU_OCC", "Occupied Housing Units Owner Occupied": "HU_OCC_OWN", "Occupied Housing Units Renter Occupied": "HU_OCC_RENT", "Housing Units": "HU_TOT", "Housing Units Vacant": "HU_VAC", "Median Value": "HH_MED_VAL", "Median Gross Rent": "HH_MED_RENT", "Median Gross Rent as a Percentage of Household  Income": "HH_MED_RENT_PCT_INC", "Median Selected Monthly Owner Costs as a  Percentage of Household Income": "HH_MED_OWN_PCT_INC", "Total Population Foreign Born": "FOREIGN_BORN", "Population 25 Years and Over High School Graduate (Includes Equivalency)": "EDU_HIGH", "Population 16 Years and Over in Labor Force in Armed Forces": "MIL_NUM"}, axis=1)

In [47]:
migpuma_acs_data

,MIGPUMA,Geo_LOGRECNO,Geo_STATE,Geo_PUMA5,TOT_POP,DENS,Area (Land),Total Population Male,Total Population Female,Total Population Under 5 Years,...,Civilian Population 18 Years and Over Veteran,Civilian Population 18 Years and Over Veteran 18 to 64 Years,Civilian Population 18 Years and Over Veteran 65 Years and Over,Civilian Population 18 Years and Over Nonveteran,Civilian Population 18 Years and Over Nonveteran 18 to 64 Years,Civilian Population 18 Years and Over Nonveteran 65 Years and Over,Total Population Native Born,FOREIGN_BORN,Total Population Foreign Born Naturalized Citizen,Total Population Foreign Born Not a Citizen
0,0100190,132,2,1500,302263,182.273226,1658.296214,150118,152145,17337,...,15386,6479,8907,223283,172049,51234,295006.0,7257.0,1940.0,5317.0
1,0100290,239,4,1303,558802,324.572600,1721.654877,275400,283402,32688,...,46991,29129,17862,384069,314684,69385,529805.0,28997.0,13689.0,15308.0
2,0100400,61,1,400,123121,148.490962,829.148107,61204,61917,6666,...,5845,2846,2999,89462,70118,19344,117038.0,6083.0,669.0,5414.0
3,0100600,63,1,600,152046,183.376165,829.148107,74066,77980,8906,...,9542,4936,4606,108249,86341,21908,146965.0,5081.0,927.0,4154.0
4,0100700,64,1,700,107102,129.171132,829.148107,52554,54548,7296,...,7282,2929,4353,74999,58851,16148,105399.0,1703.0,270.0,1433.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,7200900,153,144,1803,280389,2368.796946,118.367680,130726,149663,10791,...,7857,3431,4426,220339,166838,53501,267658.0,12731.0,6572.0,6159.0
1001,7201001,78,72,1001,124434,2123.351305,58.602644,57761,66673,4632,...,2921,1002,1919,98512,74912,23600,121739.0,2695.0,1807.0,888.0
1002,7201002,79,72,1002,156149,1104.590447,141.363707,75691,80458,7048,...,3218,1733,1485,121169,96352,24817,153632.0,2517.0,1032.0,1485.0
1003,7201101,80,72,1101,116274,559.581731,207.787341,55091,61183,4504,...,3391,647,2744,91369,69038,22331,114551.0,1723.0,1020.0,703.0


In [48]:
migpuma_acs_data["UNEMP"] = migpuma_acs_data["Population 16 Years and Over in Labor Force Civilian Unemployed"] / migpuma_acs_data["Population 16 Years and Over in Labor Force Civilian"]

In [49]:
df_ext["MIGPUMA"].value_counts()

0.0        2202915
100.0        17025
3700.0       10682
900.0         9156
400.0         8167
            ...   
51084.0         95
51085.0         87
701.0           80
6400.0          70
2601.0          62
Name: MIGPUMA, Length: 213, dtype: int64

In [50]:
df_ext['ORIGIN'].value_counts()

0603700    81968
2500390    40213
1703400    33809
0400100    31999
0800190    28154
           ...  
2202100      625
2300600      623
4806900      607
0800400      605
2201600      598
Name: ORIGIN, Length: 975, dtype: int64

In [51]:
migpuma_lodes_data["MIGPUMA"] = migpuma_lodes_data["MIGPUMA"].astype(str).str.zfill(7)

In [52]:
temp = migpuma_acs_data[["MIGPUMA", "TOT_POP", "DENS", "POP_UND_18", "POP_18_34", "POP_35_64", "POP_OVER_65", "MED_AGE", "EDU_NOHIGH", "EDU_SOMECOL", "EDU_HIGH", "EDU_BACH", "EDU_MASTERS", "EDU_PROF", "EDU_PHD", "ENRL_PUB_COLL", "ENRL_PRI_COLL", "GINI", "AGG_INC", "HH_MED_INC", "HU_OCC", "HU_OCC_OWN", "HU_OCC_RENT", "HU_TOT", "HU_VAC", "HH_MED_VAL", "HH_MED_RENT", "HH_MED_RENT_PCT_INC", "HH_MED_OWN_PCT_INC", "FOREIGN_BORN", "UNEMP", "MIL_NUM"]]
temp.columns = temp.columns.map(lambda x: str(x) + "_ORIG")
migpuma_lodes_data.columns = migpuma_lodes_data.columns.map(lambda x: str(x) + "_ORIG")
df_ext["ORIGIN"] = df_ext["ORIGIN"]
df_final = pd.merge(df_ext, temp, left_on="ORIGIN", right_on="MIGPUMA_ORIG", how="left").merge(migpuma_lodes_data, left_on = "ORIGIN", right_on="MIGPUMA_ORIG", how="left")

In [53]:
# stay_migpuma = pd.read_csv("data/stay_migpuma_2017.csv")[["ORIGIN", "HH_COST", "UNEMP", "PINCP", "EMP", "HINCP", "SMOCP", "GRNTP", "WAGP", "PERNP"]].set_index("ORIGIN")
stay_migpuma = pd.read_csv("data/stay_migpuma.csv")[["ORIGIN", "HH_COST", "UNEMP", "PINCP", "EMP", "HINCP", "SMOCP", "GRNTP", "WAGP", "PERNP"]].set_index("ORIGIN")
stay_migpuma.index = stay_migpuma.index.astype(str).str.zfill(7)

In [54]:
stay_migpuma["UNEMP_PCT"] = stay_migpuma["UNEMP"] / (stay_migpuma["EMP"] + stay_migpuma["UNEMP"])
stay_migpuma = stay_migpuma.drop(["UNEMP", "EMP"], axis=1)

In [55]:
stay_migpuma

,HH_COST,PINCP,HINCP,SMOCP,GRNTP,WAGP,PERNP,UNEMP_PCT
ORIGIN,,,,,,,,
0100190,0.139500,34000.0,55000.0,545.0,582.0,30000.0,30000.0,0.046679
0100290,0.143538,42000.0,71000.0,860.0,786.5,35000.0,38000.0,0.038005
0100400,0.138394,32000.0,49840.0,542.5,610.0,27000.0,30000.0,0.042831
0100600,0.138408,40000.0,60000.0,634.0,702.5,35000.0,36600.0,0.041121
0100700,0.134872,33000.0,53000.0,552.0,653.0,30000.0,30600.0,0.038462
...,...,...,...,...,...,...,...,...
5600100,0.156716,44650.0,72000.0,960.0,831.0,37150.0,40000.0,0.012712
5600200,0.153600,40000.0,67900.0,708.0,780.0,33000.0,35300.0,0.029255
5600300,0.177429,45000.0,80000.0,1273.0,905.0,40000.0,40000.0,0.029228


In [56]:
stay_migpuma.columns = ["HH_COST_ORIG_ADJ", "PINCP_ORIG_ADJ", "HINCP_ORIG_ADJ", "SMOCP_ORIG_ADJ", "GRNTP_ORIG_ADJ", "WAGP_ORIG_ADJ", "PERNP_ORIG_ADJ", "UNEMP_PCT_ORIG_ADJ"]

In [57]:
stay_migpuma.loc[df_final["ORIGIN"]].values

array([[1.57574089e-01, 3.70000000e+04, 7.53000000e+04, ...,
        3.50000000e+04, 3.60000000e+04, 2.79394645e-02],
       [1.59360000e-01, 3.60000000e+04, 6.00300000e+04, ...,
        3.12000000e+04, 3.31000000e+04, 4.54545455e-02],
       [1.60000000e-01, 4.00000000e+04, 6.53000000e+04, ...,
        3.50000000e+04, 3.50000000e+04, 3.98698129e-02],
       ...,
       [1.56716418e-01, 4.46500000e+04, 7.20000000e+04, ...,
        3.71500000e+04, 4.00000000e+04, 1.27118644e-02],
       [1.56716418e-01, 4.46500000e+04, 7.20000000e+04, ...,
        3.71500000e+04, 4.00000000e+04, 1.27118644e-02],
       [1.40090498e-01, 3.70000000e+04, 6.52000000e+04, ...,
        3.33000000e+04, 3.50000000e+04, 5.37261698e-02]])

In [58]:
df_final[list(stay_migpuma.columns)] = stay_migpuma.loc[df_final["ORIGIN"]].values

In [59]:
df_final

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,JOBS_EDU_NOBACH_ORIG,JOBS_EDU_BACH_ORIG,HH_COST_ORIG_ADJ,PINCP_ORIG_ADJ,HINCP_ORIG_ADJ,SMOCP_ORIG_ADJ,GRNTP_ORIG_ADJ,WAGP_ORIG_ADJ,PERNP_ORIG_ADJ,UNEMP_PCT_ORIG_ADJ
0,P,2018GQ0000049,6,1,1600,3,1,1013097,75,19,...,21563,16026,0.157574,37000.0,75300.0,1019.5,1028.0,35000.0,36000.0,0.027939
1,P,2018GQ0000058,6,1,1900,3,1,1013097,75,18,...,11048,7021,0.159360,36000.0,60030.0,819.0,846.5,31200.0,33100.0,0.045455
2,P,2018GQ0000219,6,1,2000,3,1,1013097,118,53,...,42559,30671,0.160000,40000.0,65300.0,856.5,915.0,35000.0,35000.0,0.039870
3,P,2018GQ0000246,6,1,2400,3,1,1013097,43,28,...,10800,6872,0.170087,30700.0,46630.0,623.5,687.0,28000.0,29100.0,0.071856
4,P,2018GQ0000251,6,1,2701,3,1,1013097,16,25,...,44854,31513,0.168862,35150.0,53990.0,771.5,860.0,30000.0,32000.0,0.057325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2530894,P,2018HU1400326,8,1,400,4,56,1013097,102,41,...,14622,8153,0.146434,45000.0,77460.0,959.0,850.0,42000.0,43600.0,0.018330
2530895,P,2018HU1400326,8,2,400,4,56,1013097,93,34,...,14622,8153,0.146434,45000.0,77460.0,959.0,850.0,42000.0,43600.0,0.018330
2530896,P,2018HU1400502,8,1,100,4,56,1013097,49,49,...,14447,10581,0.156716,44650.0,72000.0,960.0,831.0,37150.0,40000.0,0.012712
2530897,P,2018HU1400502,8,2,100,4,56,1013097,46,19,...,14447,10581,0.156716,44650.0,72000.0,960.0,831.0,37150.0,40000.0,0.012712


In [60]:
migpuma_lodes_data.columns[8:28]

Index(['JOBS_EARN_3334_ORIG', 'AGR_ORIG', 'EXT_ORIG', 'UTL_ORIG', 'CON_ORIG',
       'MFG_ORIG', 'WHL_ORIG', 'RET_ORIG', 'TRN_ORIG', 'INF_ORIG', 'FIN_ORIG',
       'REL_ORIG', 'PRF_ORIG', 'MNG_ORIG', 'ADM_ORIG', 'EDU_ORIG', 'MED_ORIG',
       'ENT_ORIG', 'FOD_ORIG', 'SRV_ORIG'],
      dtype='object')

In [61]:
df_final["AGR_ORIG"]

0          280
1          143
2          495
3          921
4          694
          ... 
2530894    526
2530895    526
2530896    882
2530897    882
2530898    378
Name: AGR_ORIG, Length: 2530899, dtype: int64

In [62]:
df_final["OWN_JOB_ORIG"] = 0
for occ in migpuma_lodes_data.columns[8:28]:
    key = occ
    print(occ)
    df_final["OWN_JOB_ORIG"] = np.where(df_final["NAICS"] == occ[:3], df_final[key], df_final["OWN_JOB_ORIG"])

JOBS_EARN_3334_ORIG
AGR_ORIG
EXT_ORIG
UTL_ORIG
CON_ORIG
MFG_ORIG
WHL_ORIG
RET_ORIG
TRN_ORIG
INF_ORIG
FIN_ORIG
REL_ORIG
PRF_ORIG
MNG_ORIG
ADM_ORIG
EDU_ORIG
MED_ORIG
ENT_ORIG
FOD_ORIG
SRV_ORIG


In [63]:
# for i in df_final.columns:
#     print(i)

In [64]:
migpuma_cbsa = pd.read_csv("data/migpuma_density.csv")
migpuma_cbsa

,Unnamed: 0,GISMATCH,STATEFIP,State,MIGPLAC,MIGPUMA,PWSTATE,PWPUMA,center,type,CBSA_name
0,0,600100,6,California,6,100,6,100,POINT (-2234513.518602708 322640.2378646344),T34,"San Francisco-Oakland-Hayward, CA Metro Area"
1,1,600700,6,California,6,700,6,700,POINT (-2150235.360887794 534214.0203136775),Metro,"Chico, CA Metro Area"
2,2,601100,6,California,6,1100,6,1100,POINT (-2216786.261006446 604917.4494366695),Micro,"Red Bluff, CA Micro Area"
3,3,601300,6,California,6,1300,6,1300,POINT (-2229879.805136674 352695.1396070928),T34,"San Francisco-Oakland-Hayward, CA Metro Area"
4,4,601500,6,California,6,1500,6,1500,POINT (-2096371.819538961 692307.6508446125),Metro,"Redding, CA Metro Area"
...,...,...,...,...,...,...,...,...,...,...,...
1000,1000,1304007,13,Georgia,13,4007,13,4007,POINT (1095790.60191007 -326953.0717414605),T34,"Atlanta-Sandy Springs-Roswell, GA Metro Area"
1001,1001,5151000,51,Virginia,51,51000,51,51000,POINT (1400332.713449827 105297.1155173389),Metro,"Roanoke, VA Metro Area"
1002,1002,5151001,51,Virginia,51,51001,51,51001,POINT (1519180.668946094 191928.83683416),Metro,"Charlottesville, VA Metro Area"
1003,1003,4500600,45,South Carolina,45,600,45,600,POINT (1379968.565629354 -263620.6657097285),Metro,"Columbia, SC Metro Area"


In [65]:
migpuma_cbsa["MIGPUMA_st"] = migpuma_cbsa["STATEFIP"].astype(str).str.zfill(2) + migpuma_cbsa["MIGPUMA"].astype(str).str.zfill(5)
migpuma_cbsa = migpuma_cbsa.set_index("MIGPUMA_st")
migpuma_cbsa

,Unnamed: 0,GISMATCH,STATEFIP,State,MIGPLAC,MIGPUMA,PWSTATE,PWPUMA,center,type,CBSA_name
MIGPUMA_st,,,,,,,,,,,
0600100,0,600100,6,California,6,100,6,100,POINT (-2234513.518602708 322640.2378646344),T34,"San Francisco-Oakland-Hayward, CA Metro Area"
0600700,1,600700,6,California,6,700,6,700,POINT (-2150235.360887794 534214.0203136775),Metro,"Chico, CA Metro Area"
0601100,2,601100,6,California,6,1100,6,1100,POINT (-2216786.261006446 604917.4494366695),Micro,"Red Bluff, CA Micro Area"
0601300,3,601300,6,California,6,1300,6,1300,POINT (-2229879.805136674 352695.1396070928),T34,"San Francisco-Oakland-Hayward, CA Metro Area"
0601500,4,601500,6,California,6,1500,6,1500,POINT (-2096371.819538961 692307.6508446125),Metro,"Redding, CA Metro Area"
...,...,...,...,...,...,...,...,...,...,...,...
1304007,1000,1304007,13,Georgia,13,4007,13,4007,POINT (1095790.60191007 -326953.0717414605),T34,"Atlanta-Sandy Springs-Roswell, GA Metro Area"
5151000,1001,5151000,51,Virginia,51,51000,51,51000,POINT (1400332.713449827 105297.1155173389),Metro,"Roanoke, VA Metro Area"
5151001,1002,5151001,51,Virginia,51,51001,51,51001,POINT (1519180.668946094 191928.83683416),Metro,"Charlottesville, VA Metro Area"


In [66]:
migpuma_cbsa["type_num"] = 3
migpuma_cbsa["type_num"] = np.where(migpuma_cbsa["type"] == "T34", 0, migpuma_cbsa["type_num"])
migpuma_cbsa["type_num"] = np.where(migpuma_cbsa["type"] == "Metro", 1, migpuma_cbsa["type_num"])
migpuma_cbsa["type_num"] = np.where(migpuma_cbsa["type"] == "Micro", 2, migpuma_cbsa["type_num"])

In [67]:
migpuma_cbsa.loc[df_final["ORIGIN"], "type_num"].values

array([1, 0, 1, ..., 3, 3, 3], dtype=int64)

In [68]:
df_final["TYPE_ORIG"] = migpuma_cbsa.loc[df_final["ORIGIN"], "type_num"].values

In [69]:
migpuma_cbsa = pd.read_csv('data/migpuma_density.csv')
migpuma_cbsa

,Unnamed: 0,GISMATCH,STATEFIP,State,MIGPLAC,MIGPUMA,PWSTATE,PWPUMA,center,type,CBSA_name
0,0,600100,6,California,6,100,6,100,POINT (-2234513.518602708 322640.2378646344),T34,"San Francisco-Oakland-Hayward, CA Metro Area"
1,1,600700,6,California,6,700,6,700,POINT (-2150235.360887794 534214.0203136775),Metro,"Chico, CA Metro Area"
2,2,601100,6,California,6,1100,6,1100,POINT (-2216786.261006446 604917.4494366695),Micro,"Red Bluff, CA Micro Area"
3,3,601300,6,California,6,1300,6,1300,POINT (-2229879.805136674 352695.1396070928),T34,"San Francisco-Oakland-Hayward, CA Metro Area"
4,4,601500,6,California,6,1500,6,1500,POINT (-2096371.819538961 692307.6508446125),Metro,"Redding, CA Metro Area"
...,...,...,...,...,...,...,...,...,...,...,...
1000,1000,1304007,13,Georgia,13,4007,13,4007,POINT (1095790.60191007 -326953.0717414605),T34,"Atlanta-Sandy Springs-Roswell, GA Metro Area"
1001,1001,5151000,51,Virginia,51,51000,51,51000,POINT (1400332.713449827 105297.1155173389),Metro,"Roanoke, VA Metro Area"
1002,1002,5151001,51,Virginia,51,51001,51,51001,POINT (1519180.668946094 191928.83683416),Metro,"Charlottesville, VA Metro Area"
1003,1003,4500600,45,South Carolina,45,600,45,600,POINT (1379968.565629354 -263620.6657097285),Metro,"Columbia, SC Metro Area"


In [70]:
migpuma_cbsa = pd.merge(migpuma_cbsa, cbsas[["CBSA_name", "Name_num"]], on="CBSA_name", how="left").drop_duplicates("Unnamed: 0")


In [71]:
migpuma_cbsa["GISMATCH"] = migpuma_cbsa["GISMATCH"].astype(str).str.zfill(7)
migpuma_cbsa = migpuma_cbsa.set_index("GISMATCH")
migpuma_cbsa

,Unnamed: 0,STATEFIP,State,MIGPLAC,MIGPUMA,PWSTATE,PWPUMA,center,type,CBSA_name,Name_num
GISMATCH,,,,,,,,,,,
0600100,0,6,California,6,100,6,100,POINT (-2234513.518602708 322640.2378646344),T34,"San Francisco-Oakland-Hayward, CA Metro Area",0.0
0600700,1,6,California,6,700,6,700,POINT (-2150235.360887794 534214.0203136775),Metro,"Chico, CA Metro Area",27.0
0601100,2,6,California,6,1100,6,1100,POINT (-2216786.261006446 604917.4494366695),Micro,"Red Bluff, CA Micro Area",7.0
0601300,3,6,California,6,1300,6,1300,POINT (-2229879.805136674 352695.1396070928),T34,"San Francisco-Oakland-Hayward, CA Metro Area",0.0
0601500,4,6,California,6,1500,6,1500,POINT (-2096371.819538961 692307.6508446125),Metro,"Redding, CA Metro Area",6.0
...,...,...,...,...,...,...,...,...,...,...,...
1304007,1000,13,Georgia,13,4007,13,4007,POINT (1095790.60191007 -326953.0717414605),T34,"Atlanta-Sandy Springs-Roswell, GA Metro Area",161.0
5151000,1001,51,Virginia,51,51000,51,51000,POINT (1400332.713449827 105297.1155173389),Metro,"Roanoke, VA Metro Area",479.0
5151001,1002,51,Virginia,51,51001,51,51001,POINT (1519180.668946094 191928.83683416),Metro,"Charlottesville, VA Metro Area",473.0


In [72]:
df_final["CBSA_NAME_ORIG"] = migpuma_cbsa.loc[df_final["ORIGIN"], "Name_num"].values

In [73]:
df_final["CBSA_NAME_ORIG"]

0          168.0
1          161.0
2          127.0
3           -1.0
4           48.0
           ...  
2530894     -1.0
2530895     -1.0
2530896     -1.0
2530897     -1.0
2530898     -1.0
Name: CBSA_NAME_ORIG, Length: 2530899, dtype: float64

In [74]:
df_final["MIGSP"].value_counts()

0.0     2202915
6.0       37609
48.0      28513
12.0      22431
36.0      17515
39.0      12620
17.0      12444
42.0      11970
37.0      10859
13.0      10600
53.0       9525
51.0       9349
26.0       9304
4.0        7772
34.0       7603
47.0       7248
25.0       7217
8.0        7078
18.0       7070
29.0       6735
24.0       5917
55.0       5375
27.0       5291
41.0       5232
45.0       5123
1.0        4966
21.0       4758
40.0       4449
22.0       4300
9.0        3446
49.0       3384
32.0       3286
19.0       3277
20.0       3234
5.0        3150
28.0       2724
31.0       1961
35.0       1824
16.0       1794
54.0       1659
33.0       1337
23.0       1242
11.0       1178
30.0        994
44.0        904
46.0        897
10.0        798
38.0        783
56.0        665
50.0        574
Name: MIGSP, dtype: int64

In [75]:
df_final.columns[130:286]

Index(['FAGEP', 'FANCP', 'FCITP', 'FCITWP', 'FCOWP', 'FDDRSP', 'FDEARP',
       'FDEYEP', 'FDISP', 'FDOUTP',
       ...
       'PWGTP71', 'PWGTP72', 'PWGTP73', 'PWGTP74', 'PWGTP75', 'PWGTP76',
       'PWGTP77', 'PWGTP78', 'PWGTP79', 'PWGTP80'],
      dtype='object', length=156)

In [76]:
df_final = df_final.drop(df_final.columns[130:286], axis=1)

In [77]:
# df_final.to_csv("data/us_estdata.csv")

In [78]:
df_final.to_csv("data/us_estdata_full.csv")